# Agents
- Assistant Agent  
TOOL : autogen 정보 웹서치 , 내부 데이터베이스 (회사정보)


In [19]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import StructuredMessage, TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken 
from autogen_ext.models.openai import OpenAIChatCompletionClient
import os
import dotenv

dotenv.load_dotenv()

#tool 정의
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."

async def database_search(query: str) -> str:
    """Search a database for information"""
    return "The database contains information about the company's employees."

#모델 초기화
model_client = OpenAIChatCompletionClient(
    model="gemini-2.0-flash",
    api_key= os.getenv("GOOGLE_API_KEY")
)
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[web_search, database_search], #multi-tool 사용
    system_message="""당신은 멀티에이전트 시스템에서 도움을 요청하는 사용자를 돕는 어시스턴트 에이전트입니다. 당신은 도구를 사용해서 사용자의 요청을 해결해야 합니다.
    1. 웹 검색 도구를 사용하여 오토젠의 정보를 찾아야 합니다.
    2. 데이터베이스 검색 도구를 사용하여 내부 데이터베이스에서 정보를 찾아야 합니다.""",
)

In [17]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="오토젠의 정보를 알고 싶어", source="user")],
        cancellation_token=CancellationToken(),
    )

    print(f"최종 답변: {response.chat_message.content}")


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

최종 답변: AutoGen is a programming framework for building multi-agent applications.


In [18]:
async def assistant_run2() -> None:
    response = await agent.on_messages(
        [TextMessage(content="내부 데이터베이스에서 오토젠의 정보를 알고 싶어", source="user")],
        cancellation_token=CancellationToken(),
    )

    print(f"최종 답변: {response.chat_message.content}")


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run2()

최종 답변: The database contains information about the company's employees.


# Build Tool


In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

#고정된 값을 받아야하는 경우라면?! 

# 날씨 정보 도구 함수 정의
async def get_weather(location: str, unit: str = "celsius") -> str:
    """
    주어진 위치의 현재 날씨 정보를 반환합니다.
    
    Args:
        location: 날씨 정보를 조회할 도시 또는 지역 이름
        unit: 온도 단위 (celsius 또는 fahrenheit)
    
    Returns:
        현재 날씨 정보를 포함한 문자열
    """
    # 실제로는 날씨 API를 호출해야 하지만, 예제에서는 더미 데이터 반환
    if location.lower() == "서울":
        return "서울의 현재 기온은 22°C, 맑음"
    elif location.lower() == "뉴욕":
        return "뉴욕의 현재 기온은 18°C, 흐림"
    else:
        return f"{location}의 날씨 정보를 찾을 수 없습니다."

# 모델 클라이언트 생성
model_client = OpenAIChatCompletionClient(model='gemini-2.0-flash',api_key=os.getenv("GOOGLE_API_KEY"))

# 날씨 도구를 사용하는 에이전트 생성
weather_agent = AssistantAgent(
    name="weather_assistant",
    model_client=model_client,
    tools=[get_weather],
    system_message="사용자의 날씨 관련 질문에 도구를 사용하여 답변하세요."
)

# 에이전트 사용 예시
async def run_weather_agent():
    response = await weather_agent.on_messages(
        [TextMessage(content="서울의 날씨가 어때?", source="user")],
        cancellation_token=CancellationToken()
    )
    print(response)
    print(response.chat_message.content)

await run_weather_agent()


Response(chat_message=ToolCallSummaryMessage(source='weather_assistant', models_usage=None, metadata={}, content='서울의 현재 기온은 22°C, 맑음', type='ToolCallSummaryMessage'), inner_messages=[ToolCallRequestEvent(source='weather_assistant', models_usage=RequestUsage(prompt_tokens=104, completion_tokens=5), metadata={}, content=[FunctionCall(id='', arguments='{"location":"서울"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='weather_assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='서울의 현재 기온은 22°C, 맑음', name='get_weather', call_id='', is_error=False)], type='ToolCallExecutionEvent')])
서울의 현재 기온은 22°C, 맑음


In [40]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
import os
import dotenv

dotenv.load_dotenv()

# 날씨 정보 도구 함수 정의
async def get_weather(location: str, unit: str = "celsius") -> str:
    """
    주어진 위치의 현재 날씨 정보를 반환합니다.
    """
    weather_data = {
        "서울": {"temp": 22, "condition": "맑음", "landmark": "남산타워", "culture": "K-pop의 중심지"},
        "뉴욕": {"temp": 18, "condition": "흐림", "landmark": "자유의 여신상", "culture": "문화 다양성의 도시"},
        "도쿄": {"temp": 20, "condition": "비", "landmark": "도쿄타워", "culture": "전통과 현대의 조화"},
        "파리": {"temp": 15, "condition": "맑음", "landmark": "에펠탑", "culture": "예술과 패션의 도시"}
    }
    
    location = location.lower()
    if location in weather_data:
        return weather_data[location]
    return None

class WeatherEmojiAgent:
    def __init__(self):
        self.weather_agent = AssistantAgent(
            name="weather_assistant",
            model_client=model_client,
            tools=[get_weather],
            system_message="사용자의 날씨 관련 질문에 도구를 사용하여 답변하세요."
        )
        
        self.emoji_agent = AssistantAgent(
            name="emoji_assistant",
            model_client=model_client,
            system_message="""날씨 정보를 받아서 적절한 이모지와 함께 더 풍부한 설명을 제공하세요.
            날씨 조건에 따라 다음과 같은 이모지를 사용하세요:
            - 맑음: 🌞
            - 흐림: ☁️
            - 비: 🌧️
            도시의 특징도 이모지로 표현해주세요."""
        )
    
    async def get_weather_info(self, location: str) -> str:
        # 날씨 정보 조회
        weather_response = await self.weather_agent.on_messages(
            [TextMessage(content=f"{location}의 날씨가 어때?", source="user")],
            cancellation_token=CancellationToken()
        )
        
        weather_data = await get_weather(location)
        if not weather_data:
            return f"❌ {location}의 날씨 정보를 찾을 수 없습니다."
            
        # 이모지 에이전트에게 날씨 정보 전달
        emoji_prompt = f"""
        도시: {location}
        기온: {weather_data['temp']}°C
        날씨: {weather_data['condition']}
        랜드마크: {weather_data['landmark']}
        특징: {weather_data['culture']}
        """
        
        emoji_response = await self.emoji_agent.on_messages(
            [TextMessage(content=emoji_prompt, source="user")],
            cancellation_token=CancellationToken()
        )
        
        return emoji_response.chat_message.content

async def main():
    weather_emoji_agent = WeatherEmojiAgent()
    cities = ["서울", "뉴욕", "도쿄", "파리"]
    
    print("\n🌏 전 세계 주요 도시 날씨 리포트 🌍")
    print("=" * 50)
    
    for city in cities:
        weather_info = await weather_emoji_agent.get_weather_info(city)
        print(f"\n{weather_info}")
        print("-" * 50)

if __name__ == "__main__":
    import asyncio
    await main()


🌏 전 세계 주요 도시 날씨 리포트 🌍

서울의 날씨는 🌞 맑고 기온은 22°C입니다. 남산타워가 우뚝 솟아있는 랜드마크이며 🏙️, K-pop의 중심지답게 🎤 활기찬 도시 분위기가 느껴집니다.

--------------------------------------------------

뉴욕의 날씨는 ☁️ 흐리고 기온은 18°C입니다. 자유의 여신상이 🗽 웅장하게 서 있으며, 다양한 문화가 공존하는  melting pot 🇺🇳 같은 도시입니다.

--------------------------------------------------

도쿄의 날씨는 🌧️ 비가 내리고 기온은 20°C입니다. 도쿄타워가 🗼 은은하게 빛나고 있으며, 전통적인 사찰과 현대적인 고층 빌딩이 🏯🏙️ 조화롭게 어우러진 도시입니다.

--------------------------------------------------

파리의 날씨는 🌞 맑고 기온은 15°C입니다. 에펠탑이 🗼 아름다운 자태를 뽐내고 있으며, 예술 작품과 🎨 세련된 패션 감각이 💃 넘치는 도시입니다.

--------------------------------------------------


# 필터링을 활용한 에이전트

In [60]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
import os
import json
from typing import List, Dict, Any
import dotenv

dotenv.load_dotenv()

# 쇼핑몰 데이터베이스
product_database = {
    "노트북": [
        {"id": 1, "name": "LG 그램 16", "price": 1500000, "category": "노트북", "rating": 4.7, "in_stock": True, "stock_count": 5},
        {"id": 2, "name": "맥북 프로 14", "price": 2500000, "category": "노트북", "rating": 4.9, "in_stock": True, "stock_count": 3},
        {"id": 3, "name": "삼성 갤럭시북 4", "price": 1200000, "category": "노트북", "rating": 4.5, "in_stock": True, "stock_count": 10},
        {"id": 4, "name": "MSI 게이밍 노트북", "price": 1800000, "category": "게이밍 노트북", "rating": 4.6, "in_stock": False, "stock_count": 0},
        {"id": 5, "name": "ASUS ROG 게이밍북", "price": 2200000, "category": "게이밍 노트북", "rating": 4.8, "in_stock": True, "stock_count": 2}
    ],
    "스마트폰": [
        {"id": 6, "name": "아이폰 15 Pro", "price": 1350000, "category": "스마트폰", "rating": 4.8, "in_stock": True, "stock_count": 4},
        {"id": 7, "name": "갤럭시 S23 Ultra", "price": 1450000, "category": "스마트폰", "rating": 4.7, "in_stock": True, "stock_count": 6},
        {"id": 8, "name": "픽셀 8 Pro", "price": 1200000, "category": "스마트폰", "rating": 4.6, "in_stock": True, "stock_count": 8}
    ],
    "헤드폰": [
        {"id": 9, "name": "에어팟 프로 2", "price": 350000, "category": "이어폰", "rating": 4.7, "in_stock": True, "stock_count": 15},
        {"id": 10, "name": "소니 WH-1000XM5", "price": 450000, "category": "헤드폰", "rating": 4.9, "in_stock": True, "stock_count": 7}
    ]
}

# 제품 검색 함수
async def search_products(keyword: str, min_price: float = 0, max_price: float = 10000000) -> str:

    results = []
    found_keyword = False
    
    # 카테고리 검색
    for key, products in product_database.items():
        if keyword.lower() in key.lower():
            found_keyword = True
            for product in products:
                if min_price <= product["price"] <= max_price:
                    results.append(product)
    
    # 제품명 검색
    if not found_keyword:
        for products in product_database.values():
            for product in products:
                if (keyword.lower() in product["name"].lower() and 
                    min_price <= product["price"] <= max_price):
                    results.append(product)
    
    # 검색 결과 포맷팅
    if not results:
        return f"'{keyword}' 검색 결과가 없습니다. 다른 키워드로 검색해보세요."
    
    result_str = f"'{keyword}' 검색 결과 ({len(results)}개 제품 발견):\n\n"
    for product in results:
        stock_status = f"재고 {product['stock_count']}개, " + ("재고 있음 ✅" if product["in_stock"] else "품절 ❌")
        result_str += f"- {product['name']}\n"
        result_str += f"  💰 가격: {product['price']:,}원\n"
        result_str += f"  ⭐ 평점: {product['rating']}\n"
        result_str += f"  📦 {stock_status}\n\n"
    
    return result_str

class SmartShoppingAgent:
    def __init__(self):
        self.model_client = OpenAIChatCompletionClient(
            model='gemini-2.0-flash',
            api_key=os.getenv("GOOGLE_API_KEY")
        )
        
        # 제품 검색 에이전트
        self.search_agent = AssistantAgent(
            name="search_assistant",
            model_client=self.model_client,
            tools=[search_products],
            system_message="""제품 검색 전문가입니다. 
            사용자의 요구사항을 정확히 파악하여 검색하고,
            검색 결과를 구조화된 데이터로 반환하세요.
            가격대가 지정된 경우 반드시 해당 범위 내의 제품만 검색하세요."""
        )
        
        # 분석 및 추천 에이전트
        self.recommendation_agent = AssistantAgent(
            name="recommendation_assistant",
            model_client=self.model_client,
            system_message="""쇼핑 추천 전문가입니다.
            검색된 제품들을 분석하여 다음 기준으로 추천해주세요:
            
            1. 💎 종합 평가
               - 가격, 평점, 재고 상황을 종합적으로 고려
               
            2. 💰 가성비 분석
               - 가격 대비 성능과 평점 분석
               
            3. 🎯 구매 가이드
               - 제품 선택 시 고려사항
               - 비교 장단점
               
            항상 객관적인 데이터를 기반으로 추천하고,
            사용자가 이해하기 쉽게 이모지와 함께 설명해주세요."""
        )

    async def process_query(self, user_query: str) -> str:
        # 1단계: 제품 검색
        search_response = await self.search_agent.on_messages(
            [TextMessage(content=user_query, source="user")],
            cancellation_token=CancellationToken()
        )
        
        # 검색 결과가 없는 경우
        if "검색 결과가 없습니다" in search_response.chat_message.content:
            return f"""
            😢 죄송합니다!
            
            {search_response.chat_message.content}
            
            💡 추천 검색어:
            - 카테고리로 검색: 노트북, 스마트폰, 헤드폰
            - 가격대를 지정하여 검색
            - 다른 키워드로 검색
            """
        
        # 2단계: 검색 결과 분석 및 추천
        recommendation_prompt = f"""
        다음 검색 결과를 분석하여 최적의 제품을 추천해주세요:
        
        사용자 질문: {user_query}
        
        검색 결과:
        {search_response.chat_message.content}
        
        다음 형식으로 응답해주세요:
        현재 재고 : 
        1. 💡 검색 결과 요약
        2. 🏆 최고 추천 제품
        3. 💰 가성비 추천 제품
        4. ⚠️ 구매시 고려사항
        """
        
        final_response = await self.recommendation_agent.on_messages(
            [TextMessage(content=recommendation_prompt, source="user")],
            cancellation_token=CancellationToken()
        )
        
        return final_response.chat_message.content

async def main():
    shopping_agent = SmartShoppingAgent()
    
    # 테스트 쿼리 예시
    test_queries = [
        """ 쇼핑몰에 있는 애플 또는 삼성중에 최신모델을 찾고 있어 두개의 특장점을 비교해줘"""

    ]
    
    print("\n🛍️ 스마트 쇼핑 어시스턴트 🛍️")
    print("=" * 60)
    
    for query in test_queries:
        print(f"\n👤 사용자: {query}")
        print("-" * 60)
        response = await shopping_agent.process_query(query)
        print(f"\n🤖 어시스턴트:\n{response}")
        print("=" * 60)

if __name__ == "__main__":
    await main()


🛍️ 스마트 쇼핑 어시스턴트 🛍️

👤 사용자:  쇼핑몰에 있는 애플 또는 삼성중에 최신모델을 찾고 있어 두개의 특장점을 비교해줘
------------------------------------------------------------

🤖 어시스턴트:
알겠습니다. 애플과 삼성 제품 중 어떤 종류의 제품을 찾으시는지 알려주시면, 해당 제품군의 최신 모델을 찾아 특장점을 비교 분석하여 추천해 드리겠습니다. 

예를 들어, 스마트폰을 찾으신다면 다음과 같은 형식으로 답변을 드릴 수 있습니다.

**현재 재고:** 종류를 알려주시면 확인 후 답변드리겠습니다.

1.  **💡 검색 결과 요약:**
    *   애플과 삼성의 최신 스마트폰 모델을 비교 분석합니다.
    *   각 모델의 주요 스펙, 디자인, 카메라 성능, 배터리 수명 등을 요약합니다.
2.  **🏆 최고 추천 제품:**
    *   종합적인 성능, 디자인, 사용자 경험 등을 고려하여 최고 추천 제품을 선정합니다.
    *   선정 이유와 함께 제품의 장점을 상세히 설명합니다.
3.  **💰 가성비 추천 제품:**
    *   가격 대비 성능이 뛰어난 제품을 추천합니다.
    *   주요 스펙과 장점을 설명하고, 비슷한 가격대의 다른 제품과 비교 분석합니다.
4.  **⚠️ 구매 시 고려사항:**
    *   애플과 삼성 스마트폰 선택 시 고려해야 할 사항들을 제시합니다.
    *   운영체제(iOS vs Android), 생태계, 개인적인 선호도 등을 고려하여 선택할 수 있도록 가이드를 제공합니다.

어떤 종류의 제품을 찾으시는지 알려주시면, 위와 같은 형식으로 상세하게 분석하여 최적의 제품을 추천해 드리겠습니다. 😊



### 팀 생성

## RoundRobinGroupChat
- 참가자들이 순차적으로 번갈아가며 메시지를 주고받는 팀입니다. 특정 메시지나 액션이 발생할 때까지 반복합니다.
1. `TextMentionTermination("String")` - 메시지 등장 기반 종료 조건
2. `MaxMessageTermination(int)` - 호출 횟수 기반 종료 조건
3. `ExternalTermination` - `cancellation_token.cancel()`을 활용한 종료 조건
4. `CompositeTermination` - `OrTermination`의 복합 조건

🛠️ **추가 팁**: `reset()`과 `pause()` / `resume()` 메서드 활용
- `await team.pause()` : 에이전트 실행을 중지합니다 (재개 가능)
- `await team.resume()` : 에이전트 실행을 다시 재개합니다
- `await team.reset()` : 상태를 초기화한 후 재시작합니다


In [6]:
import asyncio
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination, MaxMessageTermination


from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gemini-2.0-flash",
    api_key=os.getenv("GOOGLE_API_KEY")
)
# 기본 에이전트 생성
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="당신은 유용한 AI 어시스턴트입니다."
)

critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="건설적인 피드백을 제공하세요. 피드백이 반영되면 'APPROVE'라고 응답하세요."
)

#종료 조건 정의 - 'APPROVE' 단어가 나오면 종료
text_termination = TextMentionTermination("APPROVE")

# 예시: 최대 메시지 수가 도달하면 종료
# from autogen_agentchat.conditions import MaxMessageTermination
# text_termination = MaxMessageTermination(10)

# 예시: 외부 이벤트로 종료 제어
# from autogen_core import CancellationToken
# cancellation_token = CancellationToken()
# run_task = asyncio.create_task(
#     team.run_stream(task="...", cancellation_token=cancellation_token)
# )
# await asyncio.sleep(5)
# cancellation_token.cancel()

# 예시: 복합 조건 (10턴이거나 "DONE" 포함시 종료)
# from autogen_agentchat.conditions import OrTermination
# text_termination = OrTermination([
#     MaxMessageTermination(10),
#     TextMentionTermination("DONE")
# ])
# text_termination = TextMentionTermination("APPROVE")
#

# 팀 생성
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

# 팀 실행
# 팀 실행
async def run_story_team():
    await Console(
        team.run_stream(task="고양이가 주인공인 짧은 모험 이야기를 작성해주세요."),
        output_stats=True
    )
await run_story_team()

---------- user ----------
고양이가 주인공인 짧은 모험 이야기를 작성해주세요.


CancelledError: 

## 🎉 SelectorGroupChat - 선택 기반 팀 🌟
- 각 메시지 후 ChatCompletion 모델을 활용하여 다음 발언자를 선택하는 팀입니다. 💬


In [ ]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console

async def selector_team_example():
    model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
    
    # 각 전문가 역할을 하는 에이전트들 생성
    planner = AssistantAgent(
        "Planner",
        model_client,
        description="전체 여행 계획을 수립하는 전문가",
        system_message="당신은 여행 계획 전문가입니다. 전체적인 여행 계획과 일정을 수립하세요. 숙소나 교통에 대한 구체적인 정보가 필요한 경우 해당 전문가에게 질문하도록 유도하세요."
    )
    
    accommodation = AssistantAgent(
        "Accommodation_Expert",
        model_client,
        description="숙소 및 호텔 예약 전문가",
        system_message="당신은 숙소 예약 전문가입니다. 위치, 가격대, 시설 등 숙소에 관한 상세한 정보와 추천을 제공하세요."
    )
    
    transport = AssistantAgent(
        "Transport_Expert",
        model_client,
        description="교통 및 이동 수단 전문가",
        system_message="당신은 교통 전문가입니다. 항공, 기차, 버스, 렌터카 등 이동 수단에 관한 상세 정보와 추천을 제공하세요."
    )
    
    activities = AssistantAgent(
        "Activities_Expert",
        model_client,
        description="관광 명소 및 액티비티 전문가",
        system_message="당신은 관광 및 액티비티 전문가입니다. 방문할 만한 명소, 체험, 식당 등을 추천하고 상세 정보를 제공하세요."
    )
    
    # 맞춤형 선택기 프롬프트 작성
    selector_prompt = """
    다음 여행 상담 대화를 읽고, 현재 대화 맥락에서 가장 적합한 다음 발언자를 선택하세요:
    
    사용 가능한 전문가:
    {participants}
    
    대화 내역:
    {history}
    
    위 대화에서 다음으로 누가 발언해야 가장 적절한지 판단하여, 해당 전문가의 이름만 정확히 반환하세요.
    """
 
    # 종료 조건 설정
    termination = TextMentionTermination("PLANNING_COMPLETE")
    
    # SelectorGroupChat 팀 생성
    team = SelectorGroupChat(
        [planner, accommodation, transport, activities],
        model_client=model_client,
        selector_prompt=selector_prompt,
        termination_condition=termination,
    )
    
    # 팀 실행
    await Console(
        team.run_stream(task="서울에서 제주도로 3일간 여행 계획을 세워주세요. 활동적인 휴가를 원하며, 중급 수준의 숙소를 찾고 있습니다."),
        output_stats=True
    )

await selector_team_example()


    다음 여행 상담 대화를 읽고, 현재 대화 맥락에서 가장 적합한 다음 발언자를 선택하세요:

    사용 가능한 전문가:
    {participants}

    대화 내역:
    {history}

    위 대화에서 다음으로 누가 발언해야 가장 적절한지 판단하여, 해당 전문가의 이름만 정확히 반환하세요.
    
---------- user ----------
서울에서 제주도로 3일간 여행 계획을 세워주세요. 활동적인 휴가를 원하며, 중급 수준의 숙소를 찾고 있습니다.


# Swarm - 핸드오프 기반 팀
현재 화자가 HandOfMessage 를 보내 다음 화자를 지정 (명시)

In [5]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import Swarm
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.ui import Console
import os

async def swarm_team_example():
    model_client = OpenAIChatCompletionClient(model="gemini-2.0-flash",api_key=os.getenv("GOOGLE_API_KEY"))
    
    # 두 전문가 에이전트 생성
    math_expert = AssistantAgent(
        "Math_Expert",
        model_client=model_client,
        handoffs=["Physics_Expert"],  # 물리 전문가에게 핸드오프 가능
        system_message="당신은 수학 전문가입니다. 수학 문제를 해결할 수 있으며, 물리학적 지식이 필요한 경우 Physics_Expert에게 질문을 넘깁니다."
    )
    
    physics_expert = AssistantAgent(
        "Physics_Expert", 
        model_client=model_client,
        handoffs=["Math_Expert"],  # 수학 전문가에게 핸드오프 가능
        system_message="당신은 물리학 전문가입니다. 물리 문제를 해결할 수 있으며, 복잡한 수학적 계산이 필요한 경우 Math_Expert에게 질문을 넘깁니다."
    )
    
    # 최대 10개의 메시지 후 종료하는 조건
    termination = MaxMessageTermination(10)
    
    # Swarm 팀 생성 (수학 전문가가 첫 번째 화자)
    team = Swarm([math_expert, physics_expert], termination_condition=termination)
    
    # 팀 실행
    await Console(team.run_stream(task="물체가 100m 높이에서 떨어질 때 바닥에 도달하는 시간과 충돌 속도를 계산해주세요."))

await swarm_team_example()

---------- user ----------
물체가 100m 높이에서 떨어질 때 바닥에 도달하는 시간과 충돌 속도를 계산해주세요.
---------- Math_Expert ----------
죄송합니다. 이 문제를 해결하려면 물리학적 지식이 필요합니다. Physics_Expert에게 이 질문을 넘기겠습니다.

---------- Math_Expert ----------
[FunctionCall(id='', arguments='{}', name='transfer_to_physics_expert')]
---------- Math_Expert ----------
[FunctionExecutionResult(content='Transferred to Physics_Expert, adopting the role of Physics_Expert immediately.', name='transfer_to_physics_expert', call_id='', is_error=False)]
---------- Math_Expert ----------
Transferred to Physics_Expert, adopting the role of Physics_Expert immediately.
---------- Physics_Expert ----------
자유 낙하 운동을 가정하고 공기 저항을 무시하면 다음 공식을 사용할 수 있습니다.

시간(t)을 계산하려면:
h = (1/2) * g * t^2
여기서 h는 높이(100m), g는 중력 가속도(약 9.8m/s^2)입니다.
따라서 t = sqrt((2*h)/g)

충돌 속도(v)를 계산하려면:
v = g * t

이제 실제 계산을 위해 Math_Expert에게 질문을 넘기겠습니다.

---------- Physics_Expert ----------

---------- Physics_Expert ----------
[FunctionCall(id='', arguments='{}', name='transfer_to_math_ex

# 범용 MagenitcOneGroupChat
오케스트레이션 기반 

In [7]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console

async def magnetic_one_example():
    model_client = OpenAIChatCompletionClient(model="gemini-2.0-flash",api_key=os.getenv("GOOGLE_API_KEY"))
    
    # 분석가 에이전트
    analyzer = AssistantAgent(
        "Analyzer",
        model_client=model_client,
        system_message="당신은 자료를 분석하고 중요한 요점을 추출하는 데이터 분석가입니다."
    )
    
    # 요약가 에이전트
    summarizer = AssistantAgent(
        "Summarizer",
        model_client=model_client,
        system_message="당신은 복잡한 정보를 명확하고 간결하게 요약하는 전문가입니다."
    )
    
    # 편집자 에이전트
    editor = AssistantAgent(
        "Editor",
        model_client=model_client,
        system_message="당신은 내용의 정확성을 검토하고 문서의 품질을 향상시키는 편집자입니다."
    )
    
    # MagenticOneGroupChat 팀 생성
    team = MagenticOneGroupChat(
        [analyzer, summarizer, editor], 
        model_client=model_client,
        max_turns=15  # 최대 15턴 후 종료
    )
    
    # 팀 실행
    await Console(team.run_stream(
        task="다음 논문의 주요 내용을 분석하고 요약해주세요: '인공지능이 현대 교육에 미치는 영향과 미래 전망'"
    ))

await magnetic_one_example()

---------- user ----------
다음 논문의 주요 내용을 분석하고 요약해주세요: '인공지능이 현대 교육에 미치는 영향과 미래 전망'
---------- MagenticOneOrchestrator ----------

We are working to address the following user request:

다음 논문의 주요 내용을 분석하고 요약해주세요: '인공지능이 현대 교육에 미치는 영향과 미래 전망'


To answer this request we have assembled the following team:

Analyzer: An agent that provides assistance with ability to use tools.
Summarizer: An agent that provides assistance with ability to use tools.
Editor: An agent that provides assistance with ability to use tools.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS

*   The request is to analyze and summarize a paper titled "The Impact of Artificial Intelligence on Modern Education and Future Prospects."
*   The request is in Korean.

2.  FACTS TO LOOK UP

*   **Content of the paper "인공지능이 현대 교육에 미치는 영향과 미래 전망" ("The Impact of Artificial Intelligence on Modern Education and Future Prospects"):** The *entire* content of the paper needs to be accessed. This would ideal

Error processing publish message for MagenticOneOrchestrator_83d861e3-d1b1-478c-b11f-c9477e1d61fa/83d861e3-d1b1-478c-b11f-c9477e1d61fa
Traceback (most recent call last):
  File "/opt/anaconda3/envs/autogen/lib/python3.11/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 533, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/autogen/lib/python3.11/site-packages/autogen_core/_base_agent.py", line 113, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/autogen/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/autogen/lib/python3.11/site-packages/autogen_core/_routed_agent.py", line 485, in on_message_impl
   

CancelledError: 

In [4]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
import os

async def single_agent_loop_example():
    model_client = OpenAIChatCompletionClient(model="gemini-2.0-flash",api_key=os.getenv("GOOGLE_API_KEY"))

    # 데이터 처리 함수 정의
    async def process_data(data_chunk: str) -> str:
        """데이터 청크를 처리하고 결과를 반환합니다."""
        # 실제로는 여기서 데이터 처리 로직이 구현됩니다
        return f"처리된 데이터: {data_chunk} - 완료"

    # 데이터 처리 에이전트 생성
    processor_agent = AssistantAgent(
        "DataProcessor",
        model_client=model_client,
        tools=[process_data],
        system_message="당신은 데이터 처리 전문가입니다. 각 데이터 청크를 처리하고, 모든 처리가 완료되면 'PROCESSING_COMPLETE'라고 응답하세요.반드시 'PROCESSING_COMPLETE'라고 응답하세요."
    )

    # 텍스트 메시지 종료 조건 설정
    termination = TextMessageTermination("PROCESSING_COMPLETE")

    # 단일 에이전트 팀 생성
    team = RoundRobinGroupChat([processor_agent], termination_condition=termination)

    # 팀 실행
    await Console(
        team.run_stream(task="다음 데이터 청크를 순차적으로 처리해주세요: 'chunk1', 'chunk2', 'chunk3'. 모든 처리가 완료되면 알려주세요.")
    )

await single_agent_loop_example()


---------- user ----------
다음 데이터 청크를 순차적으로 처리해주세요: 'chunk1', 'chunk2', 'chunk3'. 모든 처리가 완료되면 알려주세요.


---------- DataProcessor ----------
[FunctionCall(id='', arguments='{"data_chunk":"chunk1"}', name='process_data')]
---------- DataProcessor ----------
[FunctionExecutionResult(content='처리된 데이터: chunk1 - 완료', name='process_data', call_id='', is_error=False)]
---------- DataProcessor ----------
처리된 데이터: chunk1 - 완료
---------- DataProcessor ----------
[FunctionCall(id='', arguments='{"data_chunk":"chunk2"}', name='process_data')]
---------- DataProcessor ----------
[FunctionExecutionResult(content='처리된 데이터: chunk2 - 완료', name='process_data', call_id='', is_error=False)]
---------- DataProcessor ----------
처리된 데이터: chunk2 - 완료
---------- DataProcessor ----------
[FunctionCall(id='', arguments='{"data_chunk":"chunk3"}', name='process_data')]
---------- DataProcessor ----------
[FunctionExecutionResult(content='처리된 데이터: chunk3 - 완료', name='process_data', call_id='', is_error=False)]
---------- DataProcessor ----------
처리된 데이터: chunk3 - 완료
---------- DataProcessor ----------
PROCESSING_COMPL

Error processing publish message for DataProcessor_d19f4b6e-6604-46ad-8245-f0d767d42671/d19f4b6e-6604-46ad-8245-f0d767d42671
Traceback (most recent call last):
  File "/opt/anaconda3/envs/autogen/lib/python3.11/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 533, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/autogen/lib/python3.11/site-packages/autogen_core/_base_agent.py", line 113, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/autogen/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/autogen/lib/python3.11/site-packages/autogen_core/_routed_agent.py", line 485, in on_message_impl
    return aw

RuntimeError: RateLimitError: Error code: 429 - [{'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.0-flash'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '36s'}]}}]
Traceback:
Traceback (most recent call last):

  File "/opt/anaconda3/envs/autogen/lib/python3.11/site-packages/autoge